In [9]:
import sys
print(sys.executable)

/workspaces/fle3d-experiment1/.venv/bin/python


In [10]:
import starfile
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

In [12]:
pytom_df = starfile.read("reconstruction_fixed_particles_pytom.star")
ri_df    = starfile.read("reconstruction_fixed_particles_ri.star")
std_df   = starfile.read("reconstruction_fixed_particles_std.star")

print("PyTom particles:", len(pytom_df))
print("RI particles:", len(ri_df))
print("STD particles:", len(std_df))

PyTom particles: 137
RI particles: 150
STD particles: 150


In [13]:
def extract_xyz(df):
    return df[
        [
            "rlnCenteredCoordinateXAngst",
            "rlnCenteredCoordinateYAngst",
            "rlnCenteredCoordinateZAngst",
        ]
    ].values

coords_pytom = extract_xyz(pytom_df)
coords_ri    = extract_xyz(ri_df)
coords_std   = extract_xyz(std_df)

In [15]:
with open("particle_locations.txt") as f:
    for i in range(10):
        print(f.readline().strip())

vesicle 425 180 95 NaN NaN NaN
vesicle 281 482 154 NaN NaN NaN
vesicle 15 10 86 NaN NaN NaN
vesicle 451 470 1 NaN NaN NaN
vesicle 43 64 74 NaN NaN NaN
fiducial 293 381 44 NaN NaN NaN
fiducial 376 328 126 NaN NaN NaN
fiducial 58 404 19 NaN NaN NaN
fiducial 319 425 54 NaN NaN NaN
fiducial 396 379 133 NaN NaN NaN


In [16]:
import pandas as pd
import numpy as np

# Load full file
gt_df = pd.read_csv(
    "particle_locations.txt",
    delim_whitespace=True,
    header=None
)

gt_df.columns = ["type", "x", "y", "z", "c4", "c5", "c6"]

print(gt_df.head())

      type    x    y    z  c4  c5  c6
0  vesicle  425  180   95 NaN NaN NaN
1  vesicle  281  482  154 NaN NaN NaN
2  vesicle   15   10   86 NaN NaN NaN
3  vesicle  451  470    1 NaN NaN NaN
4  vesicle   43   64   74 NaN NaN NaN


/tmp/ipykernel_5458/359254206.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  gt_df = pd.read_csv(


In [17]:
gt_vesicles = gt_df[gt_df["type"] == "vesicle"]

gt_xyz = gt_vesicles[["x","y","z"]].values.astype(float)

print("Ground truth vesicles:", len(gt_xyz))
print(gt_xyz[:5])

Ground truth vesicles: 5
[[425. 180.  95.]
 [281. 482. 154.]
 [ 15.  10.  86.]
 [451. 470.   1.]
 [ 43.  64.  74.]]


In [22]:
voxel_size = 15.0  # or 30.0 if binned

pytom_xyz = pytom_df[
    ["rlnCenteredCoordinateXAngst",
     "rlnCenteredCoordinateYAngst",
     "rlnCenteredCoordinateZAngst"]
].values

pytom_xyz_voxel = pytom_xyz / voxel_size

In [24]:
shape = (512,512,512)
center = np.array(shape) / 2
pytom_xyz_voxel = pytom_xyz_voxel + center

In [25]:
print("PyTom voxel min/max:", pytom_xyz_voxel.min(), pytom_xyz_voxel.max())
print("GT voxel min/max:", gt_xyz.min(), gt_xyz.max())

PyTom voxel min/max: 20.0 502.0
GT voxel min/max: 1.0 482.0


In [26]:

gt_raw = pd.read_csv(
    "particle_locations.txt",
    sep=r"\s+",
    header=None,
    names=["label","c1","c2","c3","n1","n2","n3"]
)

# keep only the class you want (VERY IMPORTANT)
# change "vesicle" -> "5mrc" or whatever the actual label is in your GT file
gt = gt_raw[gt_raw["label"].str.lower().isin(["vesicle"])][["c1","c2","c3"]].to_numpy(dtype=float)

print("GT count after label filter:", len(gt))
print("GT sample:", gt[:5])

GT count after label filter: 5
GT sample: [[425. 180.  95.]
 [281. 482. 154.]
 [ 15.  10.  86.]
 [451. 470.   1.]
 [ 43.  64.  74.]]


In [ ]:
def greedy_hits(pred_xyz, gt_xyz, max_dist=3.0):
    # simple: count GT points that have at least 1 pred within max_dist
    from scipy.spatial import cKDTree
    tree = cKDTree(pred_xyz)
    d, _ = tree.query(gt_xyz, k=1)
    return int(np.sum(d <= max_dist)), d

# pytom_xyz_voxel must already be in voxel coords (0..512)
# If not, tell me and I'll give the conversion cell again.

hits_xyz, d_xyz = greedy_hits(pytom_xyz_voxel, gt, max_dist=3)
hits_zyx, d_zyx = greedy_hits(pytom_xyz_voxel, gt[:, [2,1,0]], max_dist=3)

print("Hits assuming GT = (X,Y,Z):", hits_xyz)
print("Hits assuming GT = (Z,Y,X):", hits_zyx)

print("Median nearest dist XYZ:", np.median(d_xyz))
print("Median nearest dist ZYX:", np.median(d_zyx))

Hits assuming GT = (X,Y,Z): 0
Hits assuming GT = (Z,Y,X): 0
Median nearest dist XYZ: 115.54220008291342
Median nearest dist ZYX: 175.7526671205874


In [34]:
import starfile

star_path = "reconstruction_fixed_particles_pytom.star"  # adjust folder if needed

df = starfile.read(star_path)

print("Total detected particles:", len(df))
print("\nColumns:", df.columns.tolist())

Total detected particles: 137

Columns: ['rlnCenteredCoordinateXAngst', 'rlnCenteredCoordinateYAngst', 'rlnCenteredCoordinateZAngst', 'rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi', 'rlnLCCmax', 'rlnCutOff', 'rlnSearchStd', 'rlnTomoTiltSeriesPixelSize', 'rlnTomoName']
